In [1]:
# autoreload
%load_ext autoreload
%autoreload 2



import sys
import os

# Add the parent directory of src to the path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import torch
print(torch.__version__)
print(torch.cuda.device_count())
from unsloth import FastLanguageModel, unsloth_train

from transformers import TrainerCallback, TrainingArguments
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling, BitsAndBytesConfig
from datasets import load_dataset, Dataset
from peft import get_peft_model, LoraConfig, TaskType, PeftModel
import pprint
import json
from pathlib import Path
import transformers
import os
import numpy as np
import torch


from src.db.manager import DBManager
from src.input_to_instructions.types import InstructionQ

# 시드 설정
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    transformers.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

# 시드값 설정
seed = 3407
set_seed(seed)


2.7.0+cu128
1
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
max_seq_length = 0     # Unsloth auto supports RoPE Scaling internally!
# dtype = None              # None for auto detection
load_in_4bit = False      # Use 4bit quantization to reduce memory usage. Can be False.
device = f"cuda"



if torch.cuda.get_device_capability()[0] >= 8:
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16
# attn_implementation = "eager"
print(f"Using {attn_implementation} for attention computation.")
# QLora?

Using flash_attention_2 for attention computation.


## Load model

In [4]:
model_id = 'sh2orc/Llama-3.1-Korean-8B-Instruct'
# model_id = 'yanolja/EEVE-Korean-10.8B-v1.0'
# model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'
# model_id = 'Saxo/Linkbricks-Horizon-AI-Korean-Gemma-2-sft-dpo-27B'
# model_id = 'Bllossom/llama-3-Korean-Bllossom-70B'
# model_id = 'LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct'

model_dir = f"/model/{model_id.replace('/', '-')}"

In [5]:
# model_id = 'Bllossom/llama-3-Korean-Bllossom-70B'
# from huggingface_hub import snapshot_download
# snapshot_download(repo_id=model_id, local_dir="70b")

In [6]:
# # Tokenizer initialization
# pretrained_model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     torch_dtype=torch_dtype,
#     cache_dir=f"{model_dir}/cache",
#     # attn_implementation=attn_implementation,
#     local_files_only=True,
#     device_map="cuda"
# )

# # tokenizer = AutoTokenizer.from_pretrained(
# #     model_id,
# #     cache_dir=f"{model_dir}/cache",
# #     local_files_only=True
# # )
# # if not os.path.exists(f"{model_dir}/config.json"):
# pretrained_model.save_pretrained(model_dir)

In [7]:
# Tokenizer initialization

pretrained_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    # max_seq_length = max_seq_length,
    dtype = torch_dtype,
    load_in_4bit = False,
    load_in_8bit = False,
    # quantization_config=BitsAndBytesConfig(
    #     load_in_4bit=True,
    #     bnb_4bit_use_double_quant=True,
    #     bnb_4bit_quant_type="nf4",
    #     bnb_4bit_compute_dtype=torch_dtype
    #     # load_in_8bit=True,
    #     # llm_int8_enable_fp32_cpu_offload=False if not "27B" in model_id else True,
    # ),
    # device_map=device,
    cache_dir=f"{model_dir}/cache",
    attn_implementation=attn_implementation,
    # trust_remote_code=True
    local_files_only=True
)
tokenizer.padding_side = "left"
# tokenizer.truncation_side = "left"
print(f"Pad Token id: {tokenizer.pad_token_id} and Pad Token: {tokenizer.pad_token}")
print(f"EOS Token id: {tokenizer.eos_token_id} and EOS Token: {tokenizer.eos_token}")

# if not os.path.exists(f"{model_dir}/config.json"):
#     pretrained_model.save_pretrained(model_dir)
#     tokenizer.save_pretrained(model_dir)

==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.52.3.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.189 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu128. CUDA: 9.0. CUDA Toolkit: 12.8. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

sh2orc/Llama-3.1-Korean-8B-Instruct does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.
Pad Token id: 128004 and Pad Token: <|finetune_right_pad_id|>
EOS Token id: 128009 and EOS Token: <|eot_id|>


# Load dataset

In [8]:
train_type = [

    "WoThinking", # 0
    "WoMetadata", # 1
    "WoMetadata+Thinking", # 2

    "woQM", # 3
    "woQM+Script", # 4
    "woScript", # 5

    "woExp", # 6
    "ours" # 7
][-1]
print(f"train_type: {train_type}")

train_type: ours


In [9]:
from pathlib import Path

# BASE_DATASET_DIR = Path("../dataset/v5-250228-multimetadata")
# dataset_name = "v6-250306-optimizetoken"
dataset_name = "v7-250309-reduceinputanddatefunctioncall"
BASE_DATASET_DIR = Path(f"../finetuning/dataset/{dataset_name}")
print(f"BASE_DATASET_DIR: {BASE_DATASET_DIR}")
print(list(BASE_DATASET_DIR.iterdir()))

BASE_DATASET_DIR: ../finetuning/dataset/v7-250309-reduceinputanddatefunctioncall
[PosixPath('../finetuning/dataset/v7-250309-reduceinputanddatefunctioncall/refine.ipynb'), PosixPath('../finetuning/dataset/v7-250309-reduceinputanddatefunctioncall/scenario1'), PosixPath('../finetuning/dataset/v7-250309-reduceinputanddatefunctioncall/summary.ipynb'), PosixPath('../finetuning/dataset/v7-250309-reduceinputanddatefunctioncall/prompt.txt'), PosixPath('../finetuning/dataset/v7-250309-reduceinputanddatefunctioncall/scenario2')]


In [10]:
import re

def sub(name, common_prompt):
    # Remove the section between <|name|> ... <|name|> including the tags themselves
    # Use re.DOTALL to match newlines with '.'
    pattern = rf"\n?<\|{name}\|>[\s\S]*?<\|{name}\|>"
    common_prompt = re.sub(pattern, "", common_prompt, flags=re.DOTALL)
    return common_prompt

common_prompt = open(BASE_DATASET_DIR / f"prompt.txt", "r").read()

sub_targets = []
if train_type == "ours":
    sub_targets = []
elif train_type == "BASE":
    sub_targets = ["Thinking", "Expectation", "Mapping", "Script", "Examples"]
elif train_type in ["WoThinking"]:
    sub_targets = ["Thinking"]
elif train_type in ["woMetadata"]:
    sub_targets = ["Metadata"]
elif train_type in ["WoMetadata+Thinking"]:
    sub_targets = ["Metadata", "Thinking"]
elif train_type in ["woExp"]:
    sub_targets = ["Expectation"]

if train_type in ["woQM", "woQM+Script"]:
    sub_targets = ["QM", "Mapping"]
if train_type in ["woScript", "woQM+Script"]:
    sub_targets = ["Script"]


for sub_target in sub_targets:
    common_prompt = sub(sub_target, common_prompt)
# remove all <||>
common_prompt = re.sub(r"<\|.*?\|>", "", common_prompt)
print(common_prompt)

# print(common_prompt)

너는 유저의 HVAC 관련 질문에 답변하는 Agent의 계획을 설계하는 정확하고 훌룡한 인공지능이다. 
사용자의 질문(Input)을 받아 Agent의 Instructions를 출력해 주어야 한다.
Thinking에서는 HVAC 상식을 바탕으로 유저의 의도를 추측해 빠진 context를 추측하여 질문의 모호함을 없앤 완벽한 형태의 질문을 출력하고, 이에 대한 답변 계획을 세워야함.
Expectation에서는 유저가 기대할만한 답변을 추측해야함.
Mapping에서는 질문에 사용된 high-level taxonomy를 metadata를 바탕으로 low-level taxonomy로의 mapping을 계산하여야 함.
이때 metadata에 없는 정보를 기술하는 등 거짓된 출력을 하면 안되고, 모르는 정보가 있으면 Unknown이라 답변해야함.
Expectation의 명시된 모든 variable이 script에서 계산되야 한다.
json형식으로 출력하며, eval() 함수를 사용할 수 있도록 괄호들과 따옴표들의 순서와 닫힘을 매우 신경써서 출력해야한다.


In [11]:
scenario_dirs = [d for d in BASE_DATASET_DIR.iterdir() if d.is_dir() and "scenario" in d.name and "metadata.json" in [f.name for f in d.iterdir()]]
print(scenario_dirs)

[PosixPath('../finetuning/dataset/v7-250309-reduceinputanddatefunctioncall/scenario1'), PosixPath('../finetuning/dataset/v7-250309-reduceinputanddatefunctioncall/scenario2')]


In [12]:
from src.dateutils import normalize_sql_dates

def read_dataset(dir, path):
    # the file is originally json-list format
    # we want every first-level elements to be a string itself
    # for example, [{"Hi": "a'b'"}, {"Hi": "c'd'"}] -> ["""{"Hi": "a'b'"}""", """{"Hi": "c'd'"}"""]
    
    metadata = json.load(open(dir / "metadata.json", "r"))

    path = dir / path
    with open(path, "r", encoding="utf-8") as f:
        data = json.loads(f.read())
    
    result = []
    for d in data:
        if train_type in ["WoThinking", "WoMetadata+Thinking"]:
            del d["Response"]["Thinking"]
        elif train_type in ["woExp"]:
            del d["Response"]["Expectations"]
        
        if "Script" in d["Response"]:
            if train_type in ["woScript", "woQM+Script"]:
                new_scripts = []
                for i, script in enumerate(d["Response"]["Script"]):
                    if "data" in script:
                        new_scripts.append(script)
                d["Response"]["Script"] = new_scripts

            if train_type in ["woQM"]:
                new_scripts = []
                for i, script in enumerate(d["Response"]["Script"]):
                    if "data" in script:
                        new_scripts.append(script)
                    else:
                        for m, n in [("실내온도", "roomtemp"), ("설정온도", "settemp")]:
                            script = script.replace(f"'{m}'", f"'{n}'")
                        
                        new_scripts.append(script)
                d["Response"]["Script"] = new_scripts

            if train_type in ["woQM", "woQM+Script"]:
                mapping = d["Response"]["Mapping"]
                for i, script in enumerate(d["Response"]["Script"]):
                    if "data" not in script:
                        continue

                    t_match = re.search(r"t=('[^']+'|\[[^\]]+\])", script)
                    s_match = re.search(r"s=('[^']+'|\[[^\]]+\])", script)
                    m_match = re.search(r"m=('[^']+'|\[[^\]]+\])", script)
                    t = eval(t_match.group(1)) if t_match else None
                    s = eval(s_match.group(1)) if s_match else None
                    m = eval(m_match.group(1)) if m_match else None
                    
                    if isinstance(t, str):
                        t = [t]
                    if isinstance(s, str):
                        s = [s]
                    if isinstance(m, str):
                        m = [m]

                    t_raw = [mapping['temporal'][t_highlevel] for t_highlevel in t]
                    s_raw = [mapping['spatials'][s_highlevel] for s_highlevel in s]
                    m_raw = [mapping['modalities'][m_highlevel] for m_highlevel in m]
                    s_raw = [item for sublist in s_raw for item in (sublist if isinstance(sublist, list) else [sublist])]
                    sql = DBManager.get_query_strings_v2(
                        metadata, m_raw, t_raw, s_raw
                    )
                    sql = normalize_sql_dates(sql)
                    # replace data(...) with sql using regex
                    d["Response"]["Script"][i] = re.sub(r"data\(([^)]+)\)", lambda x: f"\"{sql}\"", script)
                del d["Response"]["Mapping"]
            #     # raise NotImplementedError
            # elif train_type in ["woOp"]:
            #     instructions = d["Response"]["Instructions"]
            #     d["Response"]["Instructions"] = [i for i in instructions if i["type"] == "q"]

        tags = d["Tags"]["Style"]

        skip_tags = ["Reason", "Graph", "Unrelated", "Prediction"]

        skip = False
        for skip_tag in skip_tags:
            if skip_tag in tags:
                skip = True
                break
        
        if skip:
            continue

        
        result.append({"Metadata": metadata, "Input": d["Input"], "Response": json.dumps(d["Response"], ensure_ascii=False)})
    # result = [{"Input": d["Input"], "Response": json.dumps(d["Response"], ensure_ascii=False)} for d in data]
    # print(f"Read {len(result)} examples from {path}")
    # print(f"Type of result: {type(result)}")
    # print(f"Type of result[0]: {type(result[0])}")
    # print(f"Type of result[0]['Input']: {type(result[0]['Input'])}")
    # print(f"Type of result[0]['Response']: {type(result[0]['Response'])}")
    return result

dataset_trs = []
dataset_tss = []
for scenario_dir in scenario_dirs:
    dataset_trs.extend(read_dataset(scenario_dir, "onlyq_tr.json"))
    dataset_tss.extend(read_dataset(scenario_dir, "onlyq_ts.json"))
    # print("Warning!!!: Test set is not mutually exclusive with training set.")

dataset_sampling_rate = 1
dataset_trs = np.random.choice(dataset_trs, size=int(len(dataset_trs) * dataset_sampling_rate), replace=False)
dataset_trs = dataset_trs.tolist()
dataset_tr = Dataset.from_list(dataset_trs) # 서로 다른 메타데이터 합치면서
dataset_ts = Dataset.from_list(dataset_tss) # Mutually exclusive한 애들은 None 됨

max_seq_length = 0
def formatting_prompts_func(examples):
    convos = []
    inputs = []
    # Iterate through each item in the batch (examples are structured as lists of values)
    for metadata, input, response in zip(examples['Metadata'], examples['Input'], examples['Response']):
        # global max_seq_length
        response.replace("    ", "")

        # print(metadata['current_datetime'])
        # print(metadata['idu_mapping'])
        answer = {
            "content": f"{response}",
            "role": "assistant"
        }
        if "llama" in model_id.lower():
            prompt = {
                "content": common_prompt,
                "role": "system"
            }
            content = ""
            if train_type not in ["WoMetadata", "WoMetadata+Thinking"]:
                content += f"Metadata:{metadata};"
            content += f"Input:{input};"
            user_input = {
                "content": content,
                "role": "user"
            }
            convos.append([prompt, user_input, answer])
            inputs.append([prompt, user_input])
        elif "gemma" in model_id.lower():
            user_input = {
                "content": f"{common_prompt};{metadata};{input}",
                "role": "user"
            }
            convos.append([user_input, answer])
        else:
            prompt = {
                "content": common_prompt,
                "role": "system"
            }
            user_input = {
                "content": f"Metadata:{metadata};Input:{input};",
                "role": "user"
            }
            convos.append([prompt, user_input, answer])
        
        # print("Answer length: ", len(response))
        # convos.append([prompt, user_input, answer])
        
        # if len(response) + 50 > max_seq_length:
        #     max_seq_length = len(response) + len(metadata) + len(input) + 50
            # print(response)
    
    texts = [
        tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
        for convo in convos]
    
    # remove \n\nCutting Knowledge Date: BLAH BLAH \nToday Date: BLAH BLAH\n\n using regex
    # texts = [re.sub(r'(\nCutting Knowledge Date:.*?\nToday Date:.*?\n\n)', '', text) for text in texts]
    return {"text": texts, "input": inputs}

dataset_tr = dataset_tr.map(formatting_prompts_func, batched=True)
dataset_ts = dataset_ts.map(formatting_prompts_func, batched=True)

max_seq_length = 10000
max_seq_length = max([len(tokenizer.encode(dataset_tr[i]['text'])) for i in range(len(dataset_tr))]) + 100
# max_seq_length += len(common_prompt)
print(max_seq_length)
print(dataset_tr[0])
print(len(dataset_tr), len(dataset_ts))
# print(f"seq length: {len(tokenizer.encode(dataset_tr[0]['text']))}")

Map:   0%|          | 0/27 [00:00<?, ? examples/s]

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

1097
{'Metadata': {'current_datetime': '2022-09-30 12:00:00', 'idu_mapping': {'01_IB5': ['우리반', '4층'], '01_IB7': ['옆반', '4층'], '02_I81': ['앞반', '4층']}, 'modality_mapping': {'roomtemp': ['실내온도'], 'settemp': ['설정온도']}}, 'Input': '지금 에너지 사용량 알려줘', 'Response': '{"Thinking": "사용자는 현재 우리반의 에너지 사용량을 알고싶어함. 현재 우리반의 에너지 사용량을 쿼리한 후 반환하면 됨.", "Expectations": ["현재 우리반의 에너지 사용량은 {{현재_우리반_에너지_사용량}}kWh입니다."], "Mapping": {"temporal": {"현재": "LAST_RECORD"}, "spatials": {"우리반": "01_IB5"}, "modalities": {"에너지 사용량": "Unknown"}}}', 'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n너는 유저의 HVAC 관련 질문에 답변하는 Agent의 계획을 설계하는 정확하고 훌룡한 인공지능이다. \n사용자의 질문(Input)을 받아 Agent의 Instructions를 출력해 주어야 한다.\nThinking에서는 HVAC 상식을 바탕으로 유저의 의도를 추측해 빠진 context를 추측하여 질문의 모호함을 없앤 완벽한 형태의 질문을 출력하고, 이에 대한 답변 계획을 세워야함.\nExpectation에서는 유저가 기대할만한 답변을 추측해야함.\nMapping에서는 질문에 사용된 high-level taxonomy를 metadata를 바탕으로 low-level taxonomy로의 mapping을 계산하여야 

In [13]:
# print(dataset_ts, )

# for t in dataset_ts['text'] + dataset_tr['text']:
#     # print(t)

    
#     # from t regex "Script": [(text)]

#     match = re.search(r'"Script": \[(.*?)\]', t)
#     if match:
#         script = match.group(1)
#         print(script)   


In [14]:
lora_r = 211
lora_alpha = lora_r * 2
lora_repr = f"v7_r{lora_r}_a{lora_alpha}_{train_type}_tr{len(dataset_tr)}_0623"
print(lora_repr)

v7_r211_a422_ours_tr27_0623


In [15]:
peft_model = FastLanguageModel.get_peft_model(
    pretrained_model,
    r=lora_r,   # LoRA rank - suggested values: 8, 16, 32, 64, 128
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", 
                    "gate_proj", "up_proj", "down_proj",
                    # "embed_tokens",
                    # "lm_head"
                    ],
    lora_alpha=lora_alpha,
    lora_dropout=0.05,   # Supports any, but = 0 is optimized
    bias="none",      # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Ideal for long context tuning
    random_state=3407,
    use_rslora=False,   # Disable rank-sensitive LoRA for simpler tasks
    loftq_config=None,   # No LoftQ, for standard fine-tuning
    max_seq_length=max_seq_length,
)
# del pretrained_model


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.6.2 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


## Training config

## Train

In [16]:
torch.cuda.empty_cache()
print(len(dataset_tr))


27


In [17]:
# import shutil
# shutil.rmtree(f"{model_dir}/chkpts/{lora_repr}", ignore_errors=True)

In [18]:
def extract_content(text: str):
    """Extract content from model output."""
    if "start_header_id" in text:
        pattern = r"<\|start_header_id\|>assistant<\|end_header_id\|>(.*?)<\|eot_id\|>"
    elif "start_of_turn" in text:
        pattern = r"<start_of_turn>model\n(.*?)<eos>"
    elif "im_start" in text:
        # <|im_start|>assistant{"Thinking": "사용자는 오늘 4층에 있는 모든 방의 설정온도의 평균값을 알고 싶어합니다. 4층에 해당하는 idu들(01_IB7, 02_I84, 02_I85)의 오늘 설정온도 데이터를 쿼리한 후 평균값을 계산하여 반환하면 됩니다.", "Expectations": ["오늘 4층의 평균 설정온도는 {{settemp_avg}}℃ 입니다."], "Instructions": [{"type": "q", "args": {"table_name": "data_t", "columns": ["settemp"], "temporal": "[DATE_TRUNC('day', DATE 'CURRENT_DATE'), DATE_TRUNC('day', DATE 'CURRENT_DATE' + INTERVAL '1 day'))", "spatials": ["01_IB7", "02_I84", "02_I85"]}, "result_name": "qr"}, {"type": "o", "script": "settemp_avg = qr['settemp'].mean();", "returns": ["settemp_avg"]}]}<|im_end|>
        pattern = r"<\|im_start\|>assistant\n(.*?)<\|im_end\|>"
    elif "|endofturn|" in text:
        pattern = r"\[\|assistant\|\](.*?)\[\|endofturn\|\]"
    
    match = re.search(pattern, text, re.DOTALL)
    result = match.group(1).strip() if match else None
    result = result.replace("<|finetune_right_pad_id|>", "")
    # result = result.replace("<|start_header_id|>", "")
    return result

In [19]:



per_device_train_batch_size, epochs = 60, 55 # 8
gradient_accumulation_steps = int(np.ceil(len(dataset_tr) / per_device_train_batch_size))
print(f"Gradient Accumulation Steps: {gradient_accumulation_steps}")

# clear all checkpoints

args = TrainingArguments(
    # num_train_epochs = 1,
    per_device_train_batch_size = per_device_train_batch_size,  # Controls the batch size per device
    per_device_eval_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps = gradient_accumulation_steps,  # Accumulates gradients to simulate a larger batch
    max_steps=gradient_accumulation_steps * epochs,
    # 리소스 제약때문에 batch size를 타협해야하는 경우가 발생 -> micro batch size를 줄이고,
 	# accumulated step을 늘려, 적절한 size로 gradient를 구해 weight update
    # https://www.youtube.com/watch?v=ptlmj9Y9iwE
    warmup_steps = gradient_accumulation_steps,
    learning_rate = 1e-4,             # Sets the learning rate for optimization
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    optim = "adamw_torch", # adamw_torch, adafactor, prodigy
    weight_decay = 0.01,              # Regularization term for preventing overfitting
    lr_scheduler_type = "cosine",  # Sets the learning rate scheduler
    seed = 3407,                        
    output_dir = f"{model_dir}/chkpts/{lora_repr}",  # Output directory for checkpoints and predictions     
    report_to = "none",              # Enables Weights & Biases (W&B) logging
    logging_steps = gradient_accumulation_steps,                # Sets frequency of logging to W&B
    logging_strategy = "steps",       # Logs metrics at each specified step
    # eval_strategy="steps",  # enable evaluation during training
    # eval_steps=gradient_accumulation_steps,
    # eval_accumulation_steps=1, # 낮을수록 eval시 사용하는 메모리 줄어듦
    # save_steps=gradient_accumulation_steps,
    # save_strategy = "steps",               
    # load_best_model_at_end = True,    # Loads the best model at the end
    # save_only_model = False,           # Saves entire model, not only weights
    # resume_from_checkpoint = f"{model_dir}/chkpts/{lora_repr}",  # Resumes training from a checkpoint
)

# metadata_str = {"idu_mapping": {"01_IB5": ["우리반", "4층"], "01_IB7": ["옆반", "4층"], "02_I81": ["앞반", "4층"]}, "modality_mapping": {"roomtemp": ["실내온도"], "settemp": ["설정온도"]}, "current_datetime": "2022-09-30 12:00:00"}
metadata_str = {
    "idu_mapping": {"01_IB5": ["옆반", "4층"], "01_IB7": ["앞반", "4층"], "02_I81": ["우리반", "4층"]},
    "modality_mapping": {"roomtemp": ["실내온도"], "settemp": ["설정온도"]},
    "current_datetime": "2022-09-29 21:30:00"
}
class CustomCallback(TrainerCallback):
    def __init__(self, tokenizer, model, eval_inputs, inputs, save_dir="./eval_generations", max_new_tokens=128):
        self.tokenizer = tokenizer
        self.model = model
        
        convos = []
        for chat in eval_inputs:
            convos.append(tokenizer.apply_chat_template(
                chat,
                tokenize=True,
                add_generation_prompt=True,
                return_tensors="pt"
            ).to(model.device))
        
        padded_inputs = []
        attention_masks = []
        
        max_length = max(cn.size(1) for cn in convos)
        for cn in convos:
            pad_length = max_length - cn.size(1)
            
            if pad_length > 0:
                # 패딩 추가
                padded = torch.cat([
                    torch.full((1, pad_length), tokenizer.pad_token_id, device=model.device),
                    cn,
                ], dim=1)
                
                # 어텐션 마스크 생성 (원본 시퀀스는 1, 패딩은 0)
                mask = torch.cat([
                    torch.zeros(1, pad_length, device=model.device),
                    torch.ones(1, cn.size(1), device=model.device),
                ], dim=1)
            else:
                padded = cn
                mask = torch.ones(1, cn.size(1), device=model.device)
            
            padded_inputs.append(padded)
            attention_masks.append(mask)
        
        # 배치 텐서 생성
        self.batch_tensor = torch.cat(padded_inputs, dim=0)
        self.attention_mask = torch.cat(attention_masks, dim=0)
        self.inputs = inputs
        self.save_dir = save_dir
        self.max_new_tokens = max_new_tokens

        os.makedirs(self.save_dir, exist_ok=True)

    def on_log(self, args, state, control, logs=None, **kwargs):
        step = state.global_step
        if step == 0:
            return  # skip at step 0
        
        loss = logs.get("loss", None)
        if loss > 0.009 or step < 45:
            return
        # inputs = self.tokenizer(
        #     self.eval_inputs, 
        #     return_tensors="pt",
        #     padding=True,
        #     # truncation=True,
        # ).to(self.model.device)


        with torch.no_grad():
            FastLanguageModel.for_inference(self.model)
            outputs = self.model.generate(
                input_ids=self.batch_tensor,
                attention_mask=self.attention_mask,
                max_new_tokens=self.max_new_tokens,
                do_sample=False,
                pad_token_id=self.tokenizer.pad_token_id,
            )
        decoded = self.tokenizer.batch_decode(
            outputs, 
            skip_special_tokens=False
        )

        # Save to file
        output_path = os.path.join(self.save_dir, f"r-{lora_repr}-step-{step}.json")
        with open(output_path, "w", encoding="utf-8") as f:
            # f.write("[\n")
            j = []
            for i, out in enumerate(decoded):
                try:
                    out = extract_content(out)
                    out = eval(out)
                except:
                    pass
                result = {
                    "Input": self.inputs[i],
                    "Scenario": "scenario2",
                    "Metadata": metadata_str,
                    "Candidate": out
                }
                j.append(result)
            f.write(json.dumps(j, ensure_ascii=False))
        FastLanguageModel.for_training(self.model)

custom_callback = CustomCallback(
    tokenizer=tokenizer,
    model=peft_model,
    eval_inputs=dataset_ts['input'],  # "input"에 prompt 또는 질문이 들어가야 합니다
    inputs=dataset_ts['Input'],
    save_dir=f"../experiments",
    max_new_tokens=1280
)
from trl import SFTTrainer

trainer = SFTTrainer(
    model = peft_model,
    processing_class = tokenizer,
    train_dataset = dataset_tr,
    # eval_dataset = dataset_ts,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,        # Can make training 5x faster for short sequences.
    args = args,
    # compute_metrics = compute_metrics,
    callbacks = [custom_callback]
)

Gradient Accumulation Steps: 1


Unsloth: Tokenizing ["text"]:   0%|          | 0/27 [00:00<?, ? examples/s]

In [20]:

trainer_stats = unsloth_train(trainer)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 27 | Num Epochs = 55 | Total steps = 55
O^O/ \_/ \    Batch size per device = 60 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (60 x 1 x 1) = 60
 "-____-"     Trainable parameters = 553,123,840/8,583,385,088 (6.44% trained)


Step,Training Loss
1,2.268800
2,2.268800
3,1.461000
4,0.835400
5,0.458500
6,0.209700
7,0.124500
8,0.093100
9,0.070700
10,0.054400


Unsloth: Will smartly offload gradients to save VRAM!


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.

TypeError: '>' not supported between instances of 'NoneType' and 'float'

In [ ]:
from IPython.display import Audio, display
import numpy as np
# Generate a 440 Hz beep for 0.2 seconds
framerate = 44100
t = np.linspace(0, 0.2, int(framerate * 0.2))
data = np.sin(2 * np.pi * 440 * t)
display(Audio(data, rate=framerate, autoplay=True))